# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from sklearn.metrics import classification_report

from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.environment import Environment
from azureml.automl.core.shared import constants

import joblib
import urllib.request
import json
import os
import ssl

## Dataset

### Overview
The dataset contains different characterstics of mushrooms, which are classified as poisonous or edible. 
Hence, in this project binary classification task will be performed. The dataset is taken from UCI ML repository and was uploaded in 2021.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-project-exp3'

experiment=Experiment(ws, experiment_name)

dataset = Dataset.get_by_name(ws, name='Mushroom Dataset')

In [3]:
train_data, test_data = dataset.random_split(percentage=0.5, seed=1)
train_data = train_data.to_pandas_dataframe()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [55]:
cluster_name = "aml-compute"
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster')
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [9]:
# TODO: Put your automl settings here
automl_settings = {
                        "iteration_timeout_minutes": 10,
                        "experiment_timeout_minutes": 20,
                        "enable_early_stopping": True,
                        "primary_metric": 'accuracy',
                        "featurization": 'auto',
                        "n_cross_validations": 5
                    }

# TODO: Put your automl config here
automl_config = AutoMLConfig(task='classification',
                            training_data=train_data,
                            label_column_name='class',
                            **automl_settings)

In [10]:
# TODO: Submit your experiment
remote_run = experiment.submit(config=automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
capstone-project-exp3,AutoML_8ba847e4-09c9-4043-8aec-0f7ed9936f98,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the

2022-03-27:18:58:26,62 WARNING  [connectionpool.py:781] Retrying (Retry(total=2, connect=3, read=2, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))': /metric/v2.0/subscriptions/baa67dbf-45d0-4d84-b662-527186361068/resourceGroups/tsetra-t332-20200511/providers/Microsoft.MachineLearningServices/workspaces/setra_analytics/runs/AutoML_8ba847e4-09c9-4043-8aec-0f7ed9936f98_15/full
2022-03-27:18:59:40,275 INFO     [explanation_client.py:334] Using default datastore for uploads


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [56]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [12]:
auto_ml_best_run, automl_model = remote_run.get_output()
print(automl_model.steps)

[('datatransformer', DataTransformer(
    task='classification',
    is_onnx_compatible=False,
    enable_feature_sweeping=True,
    enable_dnn=False,
    force_text_dnn=False,
    feature_sweeping_timeout=86400,
    featurization_config=None,
    is_cross_validation=True,
    feature_sweeping_config={}
)), ('stackensembleclassifier', StackEnsembleClassifier(
    base_learners=[('0', Pipeline(
        memory=None,
        steps=[('maxabsscaler', MaxAbsScaler(
            copy=True
        )), ('lightgbmclassifier', LightGBMClassifier(
            min_data_in_leaf=20,
            random_state=None,
            n_jobs=1,
            problem_info=ProblemInfo(
                gpu_training_param_dict={'processing_unit_type': 'cpu'}
            )
        ))],
        verbose=False
    )), ('18', Pipeline(
        memory=None,
        steps=[('standardscalerwrapper', StandardScalerWrapper(
            copy=True,
            with_mean=False,
            with_std=False
        )), ('xgboostclas

In [10]:
#Testing model on Test Data
y_predicted = automl_model.predict(test_data.to_pandas_dataframe().drop('class', axis=1))
y_test = test_data.to_pandas_dataframe()['class']

print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           e       0.95      1.00      0.98     13511
           p       1.00      0.96      0.98     16975

    accuracy                           0.98     30486
   macro avg       0.98      0.98      0.98     30486
weighted avg       0.98      0.98      0.98     30486



In [8]:
#TODO: Save the best model
joblib.dump(automl_model, 'models/automl_best_model.joblib')

['models/automl_best_model.joblib']

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [29]:
model_name = auto_ml_best_run.properties['model_name']
description = 'AutoML Mushroom Classification'
tags = None

model = remote_run.register_model(model_name = model_name, 
                                  description = description, 
                                  tags = tags)

In [32]:
auto_ml_best_run.download_file('outputs/scoring_file_v_2_0_0.py', 'inference/score.py')
auto_ml_best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')

myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")

In [41]:
inference_config = InferenceConfig(entry_script='inference/score.py', environment=myenv)

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, description='Mushroom Classification')

service = Model.deploy(ws, 'automl-mushroom', [model], inference_config, aci_config)

service.wait_for_deployment(True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-03-28 06:43:01+00:00 Creating Container Registry if not exists.
2022-03-28 06:43:01+00:00 Registering the environment.
2022-03-28 06:43:03+00:00 Building image..
2022-03-28 06:54:35+00:00 Generating deployment configuration..
2022-03-28 06:54:37+00:00 Submitting deployment to compute..
2022-03-28 06:54:43+00:00 Checking the status of deployment automl-mushroom..
2022-03-28 06:56:53+00:00 Checking the status of inference endpoint automl-mushroom.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [13]:
def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
data = {
    "Inputs": {
        "data":
        [
            {
                "cap-color": "o",
                "cap-diameter": 16.60,
                "cap-shape": "x",
                "cap-surface": "g",
                "does-bruise-or-bleed": False,
                "gill-attachment": "e",
                "gill-color": "w",
                "gill-spacing": None,
                "habitat": "d",
                "has-ring": True,
                "ring-type": "g",
                "season": "u",
                "stem-color": "w",
                "stem-height": 17.99,
                "stem-root": "s",
                "stem-surface": "y",
                "stem-width": 18.19,
                "veil-color": "w",
                "veil-type": "u"
            },
        ]
    },
    "GlobalParameters": {
        "method": "predict"
    }
}

body = str.encode(json.dumps(data))

url = 'http://b782904c-6ad8-4f1a-8b5e-5e4de099519b.westeurope.azurecontainer.io/score'
api_key = '' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'{"Results": ["p"]}'


TODO: In the cell below, print the logs of the web service and delete the service

In [52]:
print(service.get_logs())
service.delete()
model.delete()
cpu_cluster.delete()

2022-03-28T06:56:38,798208400+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2022-03-28T06:56:38,828938100+00:00 - iot-server/run 
2022-03-28T06:56:38,841281200+00:00 - rsyslog/run 
2022-03-28T06:56:38,840188800+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_4943656157e05766fedf432d8be562a4/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-03-28T06:56:39,116339200+00:00 - iot-server/finish 1 0
2022-03-28T06:56:39,117730900+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (68)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 96
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2022-03-28 06:56:41,924 | root | INFO | Starting up app insights client
logging socket was 

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
